In [1]:
import pyvista as pv
import numpy as np
import os, flopy
import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation, LinearTriInterpolator
from tqdm import tqdm

    
name = 'Model'
workspace = '../model'
mf_exe_name = '../exe/mf6'

sim = flopy.mf6.MFSimulation.load(sim_name=name, exe_name=mf_exe_name, sim_ws=workspace)

mfmodel = sim.get_model(model_name='model')

fname = os.path.join(workspace, name + '.hds')
hdobj = flopy.utils.HeadFile(fname, precision='double')
head = hdobj.get_data()
headRshp = head.reshape((head.shape[0],head.shape[2]))

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package disv...
    loading package npf...
    loading package ic...
    loading package rch...
    loading package evt...
    loading package drn...
    loading package oc...
  loading ims package model...


In [2]:
# define vertex interpolation functions
def triInterpolation(zValue, xPoints, yPoints):
    triObj = Triangulation(xPoints, yPoints)
    fz = LinearTriInterpolator(triObj, zValue)
    return fz

def interpVerticesZ(zName, zValues, xyCentroid, xyVertices):
    interpVerticesList=[]
    layTri = triInterpolation(zValues, xyCentroid[:,0], xyCentroid[:,1])
    
    for index, vertice in enumerate(tqdm(xyVertices, desc="Working vertices elevations for %s"%zName)):
        
        newZ = layTri(vertice[0],vertice[1])
        if newZ.mask:
            for index2, row2 in enumerate(cell2dList):
                if index in row2[1:]:
                    newZ = zValues[index2]
        interpVerticesList.append(newZ)
    interpVerticesArray = np.array(interpVerticesList)
    return interpVerticesArray

def interpVerticesHeads(zName, zValues, xyCentroid, xyVertices):
    interpVerticesList=[]
    #filtering over real heads
    realHeadIndexArray = (zValues>-1.00000000e+30).nonzero()[0]
    #reassigning arrays
    zValuesFiltered = zValues[realHeadIndexArray]
    xyCentroidFiltered = xyCentroid[realHeadIndexArray]
    verticesIndexFiltered = []
    for headCell in realHeadIndexArray:
        verticesIndexFiltered+=cell2dList[headCell][1:]
    verticesIndexFiltered = list(set(verticesIndexFiltered))
    #triangle interpolation function
    layTri = triInterpolation(zValuesFiltered, xyCentroidFiltered[:,0], xyCentroidFiltered[:,1])
    #empty array
    interpVerticesArray = np.ones([nvert])*-1.00000000e+30
    for vertexIndex in tqdm(verticesIndexFiltered, desc="Working vertices heads for %s"%zName):
        vertice = xyVertices[vertexIndex]
        newZ = layTri(vertice[0],vertice[1])
        if newZ.mask:
            for index2, row2 in enumerate(cell2dList):
                if vertexIndex in row2[1:]:
                    newZ = zValues[index2]
        interpVerticesArray[vertexIndex] = newZ
    return interpVerticesArray

In [3]:
# define model variables from flopy instanes
nlay = mfmodel.disv.nlay.array
nvert = mfmodel.disv.nvert.array
ncpl = mfmodel.disv.ncpl.array
modelCellZ = np.vstack((mfmodel.disv.top.array,mfmodel.disv.botm.array)) #cell Z in all layers
modelHeadZ = np.vstack([headRshp,headRshp[-1]])
# XY array for all cellvertices and centroids
verticesXYArray = np.dstack((mfmodel.disv.vertices.array.xv,mfmodel.disv.vertices.array.yv))[0] 
centroidXYArray = np.dstack((mfmodel.disv.cell2d.array.xc,mfmodel.disv.cell2d.array.yc))[0]

# XYZ for cell centroid
centroidXYZDict = {}
for lay in range(nlay):
    centroidXYZDict[str(lay)] = np.dstack((mfmodel.disv.cell2d.array.xc,mfmodel.disv.cell2d.array.yc,(modelCellZ[lay] + modelCellZ[lay+1])/2))[0]

cell2dList = []
for index, cell in enumerate(mfmodel.disv.cell2d.array):
    vertexIndexList = [x for x in list(cell)[3:] if x is not None]
    cell2dList.append(vertexIndexList)        
    
# calculate cell vertex Z
print('\nCalculating vertices elevations values')
modelVertZ = np.zeros((nlay+1,nvert))
for i in range(nlay+1):
    modelVertZ[i] = interpVerticesZ('Lay '+str(i),modelCellZ[i], centroidXYArray, verticesXYArray)
    
# calculate cell vertex head
print('\nCalculating vertices head values',flush=True)
modelVertHead = np.zeros((nlay+1,nvert))
for i in range(nlay+1):
    modelVertHead[i] = interpVerticesHeads('Lay '+str(i),modelHeadZ[i], centroidXYArray, verticesXYArray)
    
cell2dArray = np.hstack(cell2dList)


Calculating vertices elevations values


Working vertices elevations for Lay 6: 100%|██████████████████████████████████| 99154/99154 [00:09<00:00, 10429.37it/s]



Calculating vertices head values


Working vertices heads for Lay 6: 100%|███████████████████████████████████████| 99154/99154 [00:08<00:00, 11833.95it/s]


### for model geometry

In [4]:
baseGridXYZ = np.hstack((verticesXYArray,np.zeros([nvert,1])))
baseGrid = pv.PolyData(baseGridXYZ,cell2dArray)

layerTubes = {}
for lay in tqdm(range(nlay),desc=("Working geometries for model grid")):
    workingGrid = baseGrid.copy()
    workingVolume = workingGrid.extrude([0,0,1])
    for vert in range(nvert):
        workingVolume.points[vert][2] = modelVertZ[lay,vert]
        workingVolume.points[vert+nvert][2] = modelVertZ[lay+1,vert]
    layerTubes[str(lay)] = workingVolume
layerBlocks = pv.MultiBlock(layerTubes)
totalModelGrid = layerBlocks.combine()
totalModelGrid.save('../vtk/totalModelGrid.vtk')

#print('Working on the outer model shell')
workingGrid = baseGrid.copy()
workingVolume = workingGrid.extrude([0,0,1])
for vert in tqdm(range(nvert),desc=("Working on the outer model shell")):
    workingVolume.points[vert][2] = modelVertZ[0,vert]
    workingVolume.points[vert+nvert][2] = modelVertZ[nlay,vert]
workingVolume.save('../vtk/modelOuterShell.vtk')

Working on the outer model shell: 100%|███████████████████████████████████████| 99154/99154 [00:04<00:00, 22788.79it/s]


### for cell heads

In [5]:
layerTubes = {}
for lay in tqdm(range(nlay),desc=("Working head cell for model grid")):
    workingGrid = baseGrid.copy()
    #filtering cell heads
    layCellHeads = headRshp[lay]
    workingVolume = workingGrid.extrude([0,0,1])
    for vert in range(nvert):
        workingVolume.points[vert][2] = modelVertZ[lay,vert]
        workingVolume.points[vert+nvert][2] = modelVertZ[lay+1,vert]
    workingVolume.point_arrays['vertexHead']=np.hstack([modelVertHead[lay],modelVertHead[lay+1]])
    layerTubes[str(lay)] = workingVolume
layerBlocks = pv.MultiBlock(layerTubes)
totalModelGrid = layerBlocks.combine()
totalModelGrid = totalModelGrid.threshold(value=-1.00000000e+10)
totalModelGrid.save('../vtk/totalModelHeads.vtk')

Working head cell for model grid: 100%|██████████████████████████████████████████████████| 6/6 [00:24<00:00,  4.06s/it]


### for water table

In [6]:
wtCellList =[] 
for index in range(ncpl):
    headCell = headRshp[:,index]
    wtCell = headCell[headCell>-1.00000000e+30][0]
    wtCellList.append(wtCell)
wtVertexList = interpVerticesZ('Water Table',wtCellList, centroidXYArray, verticesXYArray)

workingGrid = baseGrid.copy()
workingGrid.cell_arrays['cellWaterTable']=np.array(wtCellList)
workingGrid.point_arrays['vertexWaterTable']=np.array(wtVertexList)
for vert in range(nvert):
    workingGrid.points[vert][2] = wtVertexList[vert]
workingGrid.save('../vtk/modelWaterTable.vtk')

Working vertices elevations for Water Table: 100%|█████████████████████████████| 99154/99154 [00:11<00:00, 8996.08it/s]


### for drain boundary

In [7]:

drnPkg = mfmodel.get_package('drn_0')

# get layers were the BC is located
       
drainTubes = {}
workingGrid = baseGrid.copy()
for index, drnCell in tqdm(enumerate(drnPkg.stress_period_data.array[0])):
    cellNumber = drnCell[0][1]
    cellLay = drnCell[0][0]
    #workingVol = layGeom[str(cellLay)].extract_cells(cellNumber).extract_surface()
    workingVol = workingGrid.extract_cells(cellNumber).extract_surface()
    workingVol = workingVol.extrude([0,0,1])

    cellNVerts = cell2dList[cellNumber][0] - 1
    cellVerts = cell2dList[cellNumber][1:]
        
    #print("Nro of cell verts %d"%cellNVerts)
    #print(workingVol)
    for indexVert in range(cellNVerts):
        workingVol.points[indexVert][2] = modelVertZ[cellLay,cellVerts[indexVert]]
        workingVol.points[indexVert+cellNVerts][2] = modelVertZ[cellLay+1,cellVerts[indexVert]]
    workingVol.cell_arrays['elevation'] = drnCell[1]
    workingVol.cell_arrays['cond'] = drnCell[2]
    drainTubes[str(index)] = workingVol
totalDrainsCells = pv.MultiBlock(drainTubes)
totalDrainsMerged = totalDrainsCells.combine()
totalDrainsMerged.save('../vtk/modelDrains.vtk')
    

10185it [00:57, 178.45it/s]
